In [ ]:
!pip install -q clickhouse-connect==0.6.11 transformers tokenizers torch torchvision sentencepiece protobuf tqdm

In [33]:
from transformers import pipeline
import pandas as pd
import clickhouse_connect
from tqdm import tqdm

In [7]:
client = clickhouse_connect.get_client(
    host="localhost",
    port=8123,
    username="clickhouse-user",
    password="secret"
)

In [12]:
result = client.query("SELECT id, message FROM messages")
result.result_rows[1]

('ChwKGkNJR3V5TXp5bElFREZlMEcxZ0FkTkhBS0FR',
 '@Juanito Cazares Aga caso no quiero desenfundar la Versa')

In [27]:
pipe = pipeline("text-classification", model="cardiffnlp/twitter-xlm-roberta-base-sentiment")

In [25]:
%%time
pipe(result.result_rows[1][1])
pipe("incluido.mentira, fue solo en c5n")

CPU times: total: 469 ms
Wall time: 79 ms


[{'label': 'neutral', 'score': 0.5151194930076599}]

In [34]:
results = {"id": [], "message": [], "sentiment_score": [], "sentiment_label": []}

for id, message in tqdm(result.result_rows):
    sentiment = pipe(message)[0]
    results["id"].append(id)
    results["message"].append(message)
    results["sentiment_score"].append(sentiment["score"])
    results["sentiment_label"].append(sentiment["label"])

df = pd.DataFrame(results)
df

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 35113/35113 [25:46<00:00, 22.70it/s]


,id,message,sentiment_score,sentiment_label
0,ChwKGkNOU0xrZFR5bElFREZUN1B3Z1FkYTVVRkdn,"incluido.mentira, fue solo en c5n",0.515119,neutral
1,ChwKGkNJR3V5TXp5bElFREZlMEcxZ0FkTkhBS0FR,@Juanito Cazares Aga caso no quiero desenfunda...,0.546145,negative
2,ChwKGkNPdUpsZDd3bElFREZmYkJ3Z1Fka2hNTzZR,JohnR777 VOS IGUAL SOS UN BUEN TIPO..GRACIAS.:...,0.909747,positive
3,ChwKGkNPR2p0Tjd3bElFREZYREN3Z1FkQnVnR2J3,se nota que no vivís en el conurba Hugo,0.784576,negative
4,ChwKGkNQdWc2dDd3bElFREZhc1RyUVlkYTZnT3V3,ENTONCES VAMOS CON BUKELE,0.598640,neutral
...,...,...,...,...
35108,ChwKGkNMVEsxZDd3bElFREZSY19yUVlkZ2djTVBR,roquefeker,0.378661,neutral
35109,ChwKGkNQV2xoZUx3bElFREZiQWxyUVlkOHZvR0Jn,TORINO COMAHUE,0.595542,negative
35110,ChwKGkNPTGo3T1B3bElFREZmX0R3Z1FkS2RJTzZ3,"Torinos y Falcon,imaginense los Zurdos de esta...",0.554973,negative
35111,ChwKGkNMdXk2LVh3bElFREZhMF9yUVlkbXpRQmJ3,GALIMBA HABLA,0.460618,negative


In [37]:
create_table_query = """CREATE TABLE IF NOT EXISTS sentiment
(
    id                String,
    message           String,
    sentiment_score   Float32,
    sentiment_label   String
) ENGINE = MergeTree()
ORDER BY (id);
"""

In [38]:
client.command(create_table_query)

In [39]:
rows = []
for row in df.iterrows():
    rows.append([row[1]["id"], row[1]["message"], row[1]["sentiment_score"], row[1]["sentiment_label"]])
client.insert('sentiment', rows, column_names=['id', 'message', 'sentiment_score', 'sentiment_label'])

In [43]:
client.query("SELECT id, message, sentiment_label FROM sentiment LIMIT 100").result_rows

[('ChwKGkNJLS02ZURmbElFREZjSEJ3Z1FkYzJZS3Jn',
  'PEOR ES DARLE LA MANO A GRABOY ',
  'negative'),
 ('ChwKGkNJLTB2LVhabElFREZkcUo1UWNkdVo4UDNB',
  'IZQUIERDA SON ANCIANOS ',
  'negative'),
 ('ChwKGkNJLTB4THZ3bElFREZhMF9yUVlkbXpRQmJ3',
  'DECADAS AÑOS DORADOS 90 00 ',
  'negative'),
 ('ChwKGkNJLTBoYzNTbElFREZhMF9yUVlkbXpRQmJ3', 'mmm:thinking_face:', 'neutral'),
 ('ChwKGkNJLTFuT2lfbElFREZWblJ3Z1FkSjVJRENR', ':persevering_face:', 'neutral'),
 ('ChwKGkNJLTI2LXZpbElFREZiVEN3Z1FkRHFBRHBB',
  'Basta de hablar de Milei, basta,no lo promocionen más, hablamos del peronismo,lo qué van a hacer, baaaasta con Milei...no lo levanten más no lo subestimen :folded_hands:,no sean ingenuos.basta',
  'negative'),
 ('ChwKGkNJLTJwTF9RbElFREZRREN3Z1FkRjFzUGdB',
  'La dolarización tampoco es garantía de riqueza, la propuesta de Ocampo y Cachanosky dice que tiene que venir acompañada de reformas',
  'neutral'),
 ('ChwKGkNJLTNnSVNUbElFREZRN1B3Z1FkdDQ4Qi1n',
  '@Sofia Diaz muchas veces es mas economimco aerolines 